In [ ]:
from ecmwfapi import ECMWFService

def download_wind_forecasts(start_year=2015, end_year=2015, location=(50, 10), output_file="ecmwf_wind_speed"):
    """Downloads historical ECMWF forecasts for wind speed at 100m."""

    server = ECMWFService("mars")
    lat, lon = location

    for year in range(start_year, end_year + 1):
        print(f"Retrieving wind forecast data for {year}...")

        server.execute( # https://apps.ecmwf.int/webmars/joblist/
            {
                "class": "od",
                "expver": "1",
                "stream": "oper",
                "type": "fc",
                "levtype": "sfc",
                "param": "246.228/247.228",  # Wind speed at surface level (100 m)
                "date": f"{year}-01-01/to/{year}-12-31",  # Full year in YYYY-MM-DD format
                "time": "00/12",  # Fixed time format
                "step": "0/to/144/by/3",  # Fixed step range (forecast steps from 0 to 240h)
                "grid": "0.25/0.25",  # 0.25-degree resolution grid
                "area": f"{lat}/{lon}/{lat}/{lon}",
                "format": "grib2"
            },
            target=f"{output_file}_{year}.grib"
        )

        print(f"Downloaded {output_file}_{year}.grib")

# Example Usage
download_wind_forecasts(start_year=2015, end_year=2015, location=(50, 10), output_file="ecmwf_wind_speed")


Retrieving wind forecast data for 2015...
2025-01-29 13:26:26 ECMWF API python library 1.6.3
2025-01-29 13:26:26 ECMWF API at https://api.ecmwf.int/v1
2025-01-29 13:26:27 Welcome Alexander Peters
2025-01-29 13:26:31 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web+API+FAQ or contact servicedesk@ecmwf.int
2025-01-29 13:26:32 Request submitted
2025-01-29 13:26:32 Request id: 679a1e76510c8ad1a8be8e47
2025-01-29 13:26:32 Request is submitted
2025-01-29 13:26:34 Request is active


In [4]:
import cfgrib
import xarray as xr

# File path
grib_file = "d.grib"

# Open GRIB file using cfgrib
ds = xr.open_dataset(grib_file, engine="cfgrib")

# # Print dataset information
# print(ds)

# Extract wind speed values
wind_speed = ds["u100"].values  # Change to correct variable name if different

# Print first few wind speed values
print("\nFirst few wind speed values:")
print(wind_speed)

# Print available metadata (dimensions, attributes, etc.)
print("\nDataset Metadata:")
print(ds.attrs)



First few wind speed values:
[[[[ 3.374363  ]]

  [[ 2.5454035 ]]

  [[ 2.7912197 ]]

  ...

  [[-5.505109  ]]

  [[-5.2006474 ]]

  [[-4.794712  ]]]


 [[[ 1.7817202 ]]

  [[ 3.071087  ]]

  [[ 2.889738  ]]

  ...

  [[-1.4425421 ]]

  [[ 0.1438865 ]]

  [[ 0.89628524]]]


 [[[ 4.5480423 ]]

  [[ 5.7773476 ]]

  [[ 7.646308  ]]

  ...

  [[ 2.8257227 ]]

  [[ 3.4101343 ]]

  [[ 3.2289038 ]]]


 ...


 [[[-5.283374  ]]

  [[-5.7988615 ]]

  [[-5.63908   ]]

  ...

  [[-3.3448677 ]]

  [[-4.040612  ]]

  [[-4.966055  ]]]


 [[[-4.814577  ]]

  [[-4.9538355 ]]

  [[-3.3310575 ]]

  ...

  [[-4.4196568 ]]

  [[-2.947812  ]]

  [[ 0.1106379 ]]]


 [[[-1.1155729 ]]

  [[-0.3972546 ]]

  [[-1.8707666 ]]

  ...

  [[ 3.362255  ]]

  [[ 4.386305  ]]

  [[ 4.50206   ]]]]

Dataset Metadata:
{'GRIB_edition': 1, 'GRIB_centre': 'ecmf', 'GRIB_centreDescription': 'European Centre for Medium-Range Weather Forecasts', 'GRIB_subCentre': 0, 'Conventions': 'CF-1.7', 'institution': 'European Centre for Me

<h1>9. Add weather data (reforecasts)

In [ ]:
import json
import datetime
import numpy as np
import pandas as pd
from ecmwfapi import ECMWFService
import cfgrib
import os

# Load JSON Data
with open("../data/WPPs+production.json", "r", encoding="utf-8") as f:
    wpps = json.load(f)

# ECMWF API Connection
server = ECMWFService("mars")

# Create a temporary directory for GRIB files
temp_dir = "reforecast_data"
os.makedirs(temp_dir, exist_ok=True)

# Process each wind farm
for i, wpp in enumerate(wpps):
    lat, lon = wpp["Latitude"], wpp["Longitude"]

    # Extract the earliest and latest timestamps from the data
    first_timestamp = datetime.datetime.fromisoformat(wpp["Production"][0][0]) - datetime.timedelta(days=6)
    last_timestamp = datetime.datetime.fromisoformat(wpp["Production"][-1][0])

    start_date = first_timestamp.strftime("%Y-%m-%d")
    end_date = last_timestamp.strftime("%Y-%m-%d")

    # Set a unique file name for each wind farm
    grib_file = os.path.join(temp_dir, f"{i}_{wpp['JSON-ID']}.grib")

    print(f"Retrieving wind forecast data for: {wpp['Name']} ({lat}, {lon})...")

    # Retrieve forecasts and save to file
    # job details: https://apps.ecmwf.int/mars-activity/ and https://apps.ecmwf.int/webmars/joblist/
    server.execute(
        {
            "class": "od",
            "expver": "1",
            "stream": "oper",
            "type": "fc",
            "levtype": "sfc",
            "param": "246.228/247.228",  # u and v wind components at 100m
            "date": f"{start_date}/to/{end_date}",
            "time": "00/12",
            "step": "/".join(map(str, range(0, 145, 3))),  # Forecasts up to 6 days ahead
            "area": f"{lat}/{lon}/{lat}/{lon}",
            "grid": "0.25/0.25",
            "format": "grib2"
        },
        target=grib_file  # Save the GRIB file to a temporary location
    )

    print(f"Retrieved wind forecasts for {wpp['Name']}")

Retrieving wind forecast data for: A1 DE PARC EOLIEN EN MER DE LA BAIE DE ST-BRIEUC (48.853, -2.539)...
2025-01-29 15:07:13 ECMWF API python library 1.6.3
2025-01-29 15:07:13 ECMWF API at https://api.ecmwf.int/v1
2025-01-29 15:07:14 Welcome Alexander Peters
2025-01-29 15:07:17 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web+API+FAQ or contact servicedesk@ecmwf.int
2025-01-29 15:07:18 Request submitted
2025-01-29 15:07:18 Request id: 679a3615cd19a8e20dbe9567
2025-01-29 15:07:18 Request is submitted
2025-01-29 15:07:20 Request is active


In [ ]:
grib_file = "2015_test.grib"

In [ ]:
import cfgrib
import xarray as xr

# File path
temp_dir = "reforecast_data"

# Open GRIB file using cfgrib
ds = xr.open_dataset(grib_file, engine="cfgrib")

# Extract wind speed values
wind_speed = ds["u100"].values  # Change to correct variable name if different

# Produktionsdaten laden
WPP_production = pd.read_json("data/WPPs+production.json")

for i, wpp in enumerate(wpps):
    print(f"Processing wpp {wpp['Name']}...")

    wind_speed_file = os.path.join(temp_dir, f"{i}_{wpp['JSON-ID']}.grib")

    # Windgeschwindigkeitsdaten laden
    wind_speed = xr.open_dataset(wind_speed_file, engine="cfgrib", chunks={"time": 100})
    times = pd.to_datetime(wind_speed['time'].values)
    latitudes = wind_speed['latitude'].values
    longitudes = wind_speed['longitude'].values
    u = wind_speed['u100'].values
    v = wind_speed['v100'].values


# Convert GRIB2 data to in-memory NumPy/Pandas format using cfgrib
grib_ds = cfgrib.open_datasets(grib_file)[0]  # Extract dataset
wind_speeds = grib_ds["u100"].values
times = pd.to_datetime(grib_ds.time.values)  # Extract time indices
steps = pd.to_timedelta(grib_ds.step.values)  # Lead times

# Store the extracted forecast data in a dictionary
for base_time, step, wind_speed in zip(times, steps, wind_speeds):
    forecast_time = (base_time + step).isoformat()
    forecast_data.setdefault(forecast_time, []).append(float(wind_speed))

# Assign forecasts to JSON entries
for farm in wind_farms:
    for entry in farm["Production"]:
        timestamp = entry[0]
        entry.append([])  # Initialize forecast list

        # Assign forecasts FOR that timestamp (from 6 days ago up to 0h)
        for i in range(144, -1, -3):
            forecast_time = (datetime.datetime.fromisoformat(timestamp) - datetime.timedelta(hours=i)).isoformat()
            if forecast_time in forecast_data:
                entry[2].append(np.mean(forecast_data[forecast_time]))  # Take mean if multiple values exist
            else:
                entry[2].append(None)  # Placeholder if missing data

# Save updated JSON
with open("data/WPPs+production+wind_hist.json", "w", encoding="utf-8") as f:
    json.dump(wind_farms, f, indent=4)

print("Processing complete. Updated JSON saved.")

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.interpolate import interp2d
import warnings
import json
import xarray as xr
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Basisverzeichnisse für Input und Output
input_dir = r"C:\Users\alexa\Documents\Webapp\data\weather_history"
output_dir = "data"

# Produktionsdaten laden
WPP_production = pd.read_json("data/WPPs+production.json")

# Schleife über die Jahre 2015 bis 2024
for year in range(2015, 2025):
    print(f"Processing year {year}...")

    # Dateinamen für Input- und Output-Dateien
    wind_speed_file = os.path.join(input_dir, f"{year}.grib")
    output_file_json = os.path.join(output_dir, f"WPPs+production+wind_{year}.json")
    output_file_excel = os.path.join(output_dir, f"WPPs+production+wind_{year}.xlsx")

    # Windgeschwindigkeitsdaten laden
    wind_speed = xr.open_dataset(wind_speed_file, engine="cfgrib", chunks={"time": 100})
    times = pd.to_datetime(wind_speed['time'].values)
    latitudes = wind_speed['latitude'].values
    longitudes = wind_speed['longitude'].values
    u = wind_speed['u100'].values
    v = wind_speed['v100'].values

    WPP_production_wind = []

    # Iteration über alle Windkraftwerke
    for i, wpp in WPP_production.iterrows():
        lon = wpp['Longitude']
        lat = wpp['Latitude']
        production = wpp['Production']

        # Filtere Produktionsdaten für das aktuelle Jahr
        production_subset = [entry for entry in production if str(year) in entry[0]]

        if not production_subset:
            print(f"Wind power plant {i+1}/{len(WPP_production)} has no production data for {year}, skipping...")
            continue
        else:
            print(f"Wind power plant {i+1}/{len(WPP_production)} for year {year}")
            interpolated_production = []
            for j, entry in enumerate(production_subset):
                time_str, production_value = entry
                timestep = pd.to_datetime(time_str)
                if timestep in times:
                    time_index = times.get_loc(timestep)

                    wind_speeds = np.sqrt(u[time_index]**2 + v[time_index]**2)
                    spatial_interpolator = interp2d(longitudes, latitudes, wind_speeds, kind='linear')
                    wind_speed_value = spatial_interpolator(lon, lat)[0]
                    wind_speed_value = round(wind_speed_value, 2)

                    interpolated_production.append([time_str, production_value, wind_speed_value])

            # Produktionsdaten aktualisieren
            wpp['Production'] = interpolated_production
            WPP_production_wind.append(wpp.to_dict())

    # Speichere die aktualisierten Produktionsdaten
    with open(output_file_json, 'w', encoding='utf-8') as json_file:
        json.dump(WPP_production_wind, json_file, ensure_ascii=False, indent=4)
    print(f"Updated JSON file for {year} saved to: {output_file_json}")

    # Konvertiere die Liste in einen DataFrame und speichere als Excel
    df_WPP_production = pd.DataFrame(WPP_production_wind)
    df_WPP_production.to_excel(output_file_excel, index=False)
    print(f"Updated Excel file for {year} saved to: {output_file_excel}")